In [1]:
import itertools
import matplotlib.pyplot as plt
import numpy as np
import itertools

from ipywidgets import interactive
import mplcursors
from functools import lru_cache
# %matplotlib notebook

In [2]:
# Define the calculate_iou() function
def calculate_miou(tp, fp, tn, fn):
    iou_pos = tp / (tp + fp + fn)
    iou_neg = tn / (tn + fp + fn)
    miou = (iou_pos + iou_neg) / 2
    return miou

def calculate_accuracy(TP,FP,TN,FN):
    return (TP+TN)/(TP+TN+FP+FN)

def calculate_mean_accuracy(TP,FP,TN,FN):
    return (calculate_accuracy(TP,FP,TN,FN)+calculate_accuracy(TN, FN, TP, FP))/2

# Define the generate_combinations() function
@lru_cache(5, False)
def generate_combinations(num_positives, num_negatives):
    # Generate all possible combinations of TP, FP, TN, and FN
    combinations = list(itertools.product(np.arange(0,max(num_positives,num_negatives)+1), repeat=4))
    # Filter out invalid combinations
    valid_combinations = []
    for c in combinations:
        if sum(c) == (num_positives+num_negatives) and max(c) <= max(num_positives,num_negatives):
            valid_combinations.append(c)
    return valid_combinations

# Define the plot_iou() function
def plot_iou(num_positives, num_negatives):
    valid_combinations = generate_combinations(num_positives, num_negatives)
    ious = []
    for c in valid_combinations:
        iou = calculate_miou(*c)
        ious.append(iou)

    # Plot the results as a bar chart
    labels = [str(c) for c in valid_combinations]
    x_pos = [i for i, _ in enumerate(labels)]
    fig, ax = plt.subplots(figsize=(8, 6))
    ax.bar(x_pos, ious, color='green')
    ax.set_xlabel("Combination of TP, FP, TN, and FN")
    ax.set_ylabel("mIoU")
    ax.set_title(f"mIoU for {num_positives} positive labels and {num_negatives} negative labels")
    ax.set_xticks(x_pos)
    ax.set_xticklabels(labels, rotation='vertical')

    # Add annotations to the plot
    mplcursors.cursor(ax.bar(x_pos, ious), hover=True).connect(
        "add", lambda sel: sel.annotation.set_text(f"mIoU: {sel.target[1]:.3f}"))

    plt.show()

num_positives, num_negatives = 45,55
print("tp, fp, tn, fn")
# plot_iou(num_positives, num_negatives)

tp, fp, tn, fn


In [ ]:
generate_combinations(num_positives, num_negatives)

In [ ]:
# Define the find_iou_combination() function
def find_iou_combination(mIoU, num_positives, num_negatives):
    # Generate all valid combinations of TP, FP, TN, and FN
    valid_combinations = generate_combinations(num_positives, num_negatives)

    # Calculate the mIoU for each combination and find the closest matches
    closest_matches = []
    closest_diff = float('inf')
    for c in valid_combinations:
        iou = calculate_miou(*c)
        diff = abs(iou - mIoU)
        if diff < closest_diff:
            closest_diff = diff
            closest_matches = [(c, iou)]
        elif diff == closest_diff:
            closest_matches.append((c, iou))

    return closest_matches


# Call the find_iou_combination() function with mIoU=0.5, num_positives=5, and num_negatives=5
mIoU = 0.3196
result = find_iou_combination(mIoU, num_positives, num_negatives)

print("tp fp tn fn")
for r in result: print(r)

In [ ]:
tp,fp,tn,fn = 4,96,0,0
print("all pos")
print(calculate_miou(tp,fp,tn,fn))
# print(calculate_mean_accuracy(tp,fp,tn,fn))
print("all neg")
print(calculate_miou(fp,tp,fn,tn))
# print(calculate_mean_accuracy(fp,tp,fn,tn))

from sklearn.metrics import jaccard_score
target = [1]*71 + [0]*29
preds = [1]*100
keepIoU, discardIoU = jaccard_score(target, preds, average=None)
mIoU = jaccard_score(target, preds, average='macro')
print(f"keepIOU {keepIoU} discardIoU {discardIoU} miou {mIoU} ")

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
%matplotlib notebook

def get_tp_tn(num_positives, num_negatives):
    valid_combinations = generate_combinations(num_positives, num_negatives)
    TP = [t[0] for t in valid_combinations]
    TN = [t[2] for t in valid_combinations]
    MIOU = [calculate_miou(*t) for t in valid_combinations]
    return np.array(TP),np.array(TN),np.array(MIOU)

# Define the variables and the score
x, y, z = get_tp_tn(num_positives,num_negatives)

# Create a 3D figure and axis
fig = plt.figure(figsize=(8, 6))
ax = fig.add_subplot(111, projection='3d')

# Add the points as a scatter plot
ax.scatter(x, y, z, c=z, cmap='viridis')

In [ ]:
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.pyplot as plt
from matplotlib import cm
import numpy as np

fig = plt.figure()
ax = Axes3D(fig)
surf = ax.plot_trisurf(x, y, z, cmap=cm.jet, linewidth=0.1)
ax.plot_trisurf([0,10,0,10],[0,0,10,10],[.5,.5,.5,.5], alpha=.9)
fig.colorbar(surf, shrink=0.5, aspect=5)
ax.set_xlabel('TP')
ax.set_ylabel('TN')
ax.set_zlabel('mIoU')
# plt.savefig('teste.pdf')
plt.show()